In [7]:
import pandas as pd
import numpy as np
X_train = pd.read_csv("data/x_train.txt",header=None,sep=" ")
y_train = pd.read_csv("data/y_train.txt",header=None,sep=" ")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import f_classif
import matplotlib.pyplot as plt
import seaborn as sns
f_values, p_values = f_classif(X_train, y_train)


anova_df = pd.DataFrame({
    'Feature': range(len(p_values)),  # numer cechy
    'p_values': p_values
})
anova_df.sort_values('p_values',ascending=False)

C:\Users\Michał\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Feature,p_values
111,111,0.999768
296,296,0.999151
448,448,0.997797
53,53,0.997533
466,466,0.994402
...,...,...
459,459,0.067997
158,158,0.062945
253,253,0.059561
109,109,0.053987


In [27]:
y_train

,0
0,1
1,0
2,0
3,1
4,1
...,...
4995,0
4996,0
4997,0
4998,1
